In [1]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from read_data import get_matches_from_dir
from enums import dan_names_dict

In [2]:
import read_data # weird notebook workaround
notebook_pardir = os.path.dirname(read_data.__file__)
notebook_pardir

'/Users/danielrjohnson/My_Drive/Programming/Personal/Tekken8-Match-Stats'

In [3]:
start_date = "01_01_24"
end_date = None
df, failure_count = get_matches_from_dir(start_date=start_date, end_date=end_date)
print(f"{round((failure_count / len(df)) * 100, 2)}% of files failed to read")
print(df.columns)
df.to_csv(os.path.join(notebook_pardir, "plots", "data", "raw_matches.csv"), index=False)
df.head()

0.15% of files failed to read
Index(['battleId', 'battleType', 'gameVersion', 'winResult', 'totalRoundNum',
       'battleAt', 'viewNum', 'stageId', 'highlightFlag', '1pUserId',
       '1pPlayerName', '1pPolarisId', '1pOnlineId', '1pNgWordFlag',
       '1pPlatform', '1pRank', '1pTekkenPower', '1pCharaId', '1pWinRoundNum',
       '1pTagType01', '1pTagType02', '1pTagType03', '2pUserId', '2pPlayerName',
       '2pPolarisId', '2pOnlineId', '2pNgWordFlag', '2pPlatform', '2pRank',
       '2pTekkenPower', '2pCharaId', '2pWinRoundNum', '2pTagType01',
       '2pTagType02', '2pTagType03'],
      dtype='object')


,battleId,battleType,gameVersion,winResult,totalRoundNum,battleAt,viewNum,stageId,highlightFlag,1pUserId,...,2pOnlineId,2pNgWordFlag,2pPlatform,2pRank,2pTekkenPower,2pCharaId,2pWinRoundNum,2pTagType01,2pTagType02,2pTagType03
0,00610fc8c57942f8bde15db45c5ac8c2,4,10104,1,4,1708008383,0,stg_1100,False,130213220808144959,...,76561198810119667,0,3,5,75613,chr_0035,1,0,0,0
1,007e06c244fa4d3799124ada884da8b2,2,10104,1,5,1708008377,0,stg_1600,False,472839240201025404,...,76561198032791159,0,3,10,117569,chr_0035,2,0,0,0
2,00c146d70b1647cd8bacdc0929cca847,2,10104,2,5,1708008364,0,stg_0200,False,124522190911025710,...,76561198298460427,0,3,22,181001,chr_0004,3,0,0,0
3,00d390f45cb84aee972457d1a52cf1ea,2,10104,2,3,1708008372,0,stg_0700,False,275938240128083408,...,76561198958696626,0,3,18,144681,chr_0032,3,0,0,0
4,01183af7d37a49d89716a9c0eac9214a,2,10104,2,3,1708008385,0,stg_1600,False,122358240126104819,...,76561198053444753,0,3,20,175289,chr_0034,3,0,0,0


In [4]:
cols_1p_normalized = ["battleId", "battleType", "gameVersion", "winResult", "totalRoundNum", "battleAt", "stageId",
                        "1pUserId", "1pPlayerName", "1pPlatform", "1pRank", "1pTekkenPower", "1pCharaId", "1pWinRoundNum"]

def mirror_row(row):
    row["winResult"] = 1 if row["winResult"] == 2 else 2 # swap win if 2p win
    for feature in ["UserId", "PlayerName", "Platform", "Rank", "TekkenPower", "CharaId", "WinRoundNum"]:
        row[f"1p{feature}"] = row[f"2p{feature}"]
    return row[cols_1p_normalized]

df_1p_normalized = pd.concat([df[cols_1p_normalized], df.copy().apply(mirror_row, axis=1)])
df_1p_normalized.to_csv(os.path.join(notebook_pardir, "plots", "data", "matches_1p_normalized.csv"), index=False)
df_1p_normalized.head()

,battleId,battleType,gameVersion,winResult,totalRoundNum,battleAt,stageId,1pUserId,1pPlayerName,1pPlatform,1pRank,1pTekkenPower,1pCharaId,1pWinRoundNum
0,00610fc8c57942f8bde15db45c5ac8c2,4,10104,1,4,1708008383,stg_1100,130213220808144959,쑤우니치입,3,0,7162,chr_0006,3
1,007e06c244fa4d3799124ada884da8b2,2,10104,1,5,1708008377,stg_1600,472839240201025404,Seye,3,13,92052,chr_0036,3
2,00c146d70b1647cd8bacdc0929cca847,2,10104,2,5,1708008364,stg_0200,124522190911025710,Doby,3,23,190512,chr_0018,2
3,00d390f45cb84aee972457d1a52cf1ea,2,10104,2,3,1708008372,stg_0700,275938240128083408,ＡＧＡＳＵ,8,19,149277,chr_0011,0
4,01183af7d37a49d89716a9c0eac9214a,2,10104,2,3,1708008385,stg_1600,122358240126104819,恋々@鮪一家,8,19,147790,chr_0017,0


In [5]:
player_info = defaultdict(dict)

for uid, matches in df_1p_normalized.groupby("1pUserId"):
    for char, charmatches in matches.groupby("1pCharaId"):
        most_recent_game = charmatches.loc[charmatches["battleAt"].idxmax()]

        player_info[uid+char].update({
            "user_id": uid,
            "last_seen": most_recent_game["battleAt"],
            "username": most_recent_game["1pPlayerName"],
            "platform": most_recent_game["1pPlatform"],
            "rank_id": most_recent_game["1pRank"],
            "tekken_power": most_recent_game["1pTekkenPower"],
            "character_id": most_recent_game["1pCharaId"]
        })

df_player = pd.DataFrame(player_info).transpose().reset_index(drop=True)
df_player.to_csv(os.path.join(notebook_pardir, "plots", "data", "player_info.csv"), index=False)
df_player.head()

,user_id,last_seen,username,platform,rank_id,tekken_power,character_id
0,000001220902102523,1708009638,Rukoraku,9,14,102716,chr_0033
1,000001220902102523,1708012375,Rukoraku,9,8,102716,chr_0035
2,000003240206072109,1709364735,CookthemanEggs,8,14,107224,chr_0006
3,000008201007222404,1709315410,OniEUW,3,14,112413,chr_0011
4,000017200602140341,1709355259,초장동불꽃주먹,3,15,115112,chr_0015


In [6]:
for rank in list(dan_names_dict.keys())[:-1]:
    matchup_info = []

    for char, chardata in df[df["1pRank"] == rank].groupby("1pCharaId"): # 1p of given character vs others
        for otherchar, matches in chardata.groupby("2pCharaId"):
            matches = matches[matches["1pCharaId"] != matches["2pCharaId"]] # remove mirrors
            n_games = len(matches)
            n_wins = len(matches[matches["winResult"] == 1])
            win_rate = n_wins / n_games if n_games != 0 else np.NaN
            avg_rounds = matches["totalRoundNum"].mean()
            matchup_info.append([char, otherchar, n_games, n_wins, win_rate, avg_rounds])

    for char, chardata in df[df["2pRank"] == rank].groupby("2pCharaId"): # 2p of given character vs others
        for otherchar, matches in chardata.groupby("1pCharaId"):
            matches = matches[matches["1pCharaId"] != matches["2pCharaId"]] # remove mirrors
            n_games = len(matches)
            n_wins = len(matches[matches["winResult"] == 2])
            win_rate = n_wins / n_games if n_games != 0 else np.NaN
            avg_rounds = matches["totalRoundNum"].mean()
            matchup_info.append([char, otherchar, n_games, n_wins, win_rate, avg_rounds])
            
    df_matchup = pd.DataFrame(matchup_info, columns=["char", "other_char", "n_games", "n_wins", "win_rate", "avg_rounds"])
    # entries for each char are currently separate, groupby to join them
    df_matchup = df_matchup.groupby(by=["char", "other_char"]).agg(
        {"n_games": "sum", "n_wins": "sum", "win_rate": "mean", "avg_rounds": "mean"}).reset_index()
    df_matchup.to_csv(os.path.join(notebook_pardir, "plots", "data", "matchup_data", 
                                   f"matchup_info_dan={rank}.csv"), index=False)